In [1]:
import os
import gzip
from scipy import stats
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import networkx as nx
from sklearn.metrics import roc_curve, auc, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import scienceplots
import requests
import pickle
import powerlaw
import gc
import math
import random

from main_function import *
import main_function
from importlib import reload
reload(main_function)

<module 'main_function' from '/home/qinjiajie/jupyter_code/AUC_upload/main_function.py'>

# n=3000, L=4500 as an example

## generating ground-truth

In [2]:
nodes = 3000
md = 3

ga_inde = np.array(list(range(160, 260, 20))) / 100

ground_truth = dict()

for k in range(5):
    z=0
    gg = dict()
    for ga in ga_inde:
        uu = generate_ground_nograph(nodes, ga,md,z)
        G_temp  = uu[0]
        z = uu[1]
        gg['%s' %ga] = G_temp
        print('gamma=%s complete' % ga)
    ground_truth['%s' % k] = gg

ga_inde = np.array(list(range(170, 270, 20))) / 100

for k in range(5,10,1):
    z=0
    gg = dict()
    for ga in ga_inde:
        uu = generate_ground_nograph(nodes, ga,md,z)
        G_temp  = uu[0]
        z = uu[1]
        gg['%s' %ga] = G_temp
        print('gamma=%s complete' % ga)
    ground_truth['%s' % k] = gg

Calculating best minimal value for power law fit
gamma=1.6 complete
Calculating best minimal value for power law fit
gamma=1.8 complete
Calculating best minimal value for power law fit
gamma=2.0 complete
Calculating best minimal value for power law fit
gamma=2.2 complete
Calculating best minimal value for power law fit
gamma=2.4 complete
Calculating best minimal value for power law fit
gamma=1.6 complete
Calculating best minimal value for power law fit
gamma=1.8 complete
Calculating best minimal value for power law fit
gamma=2.0 complete
Calculating best minimal value for power law fit
gamma=2.2 complete
Calculating best minimal value for power law fit
gamma=2.4 complete
Calculating best minimal value for power law fit
gamma=1.6 complete
Calculating best minimal value for power law fit
gamma=1.8 complete
Calculating best minimal value for power law fit
gamma=2.0 complete
Calculating best minimal value for power law fit
gamma=2.2 complete
Calculating best minimal value for power law fit

In [4]:
file_path = './result_set/ground_truth_3000_3.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(ground_truth, f)

## inference

In [5]:
file_path = './result_set/ground_truth_3000_3.pkl'
with open(file_path, 'rb') as f:
    ground_truth = pickle.load(f)

In [ ]:
M=10
alpha=0.5
beta=0.3
# result1 = dict()

for mk in list(ground_truth.keys())[:1]:
    a = dict()
    a_s = parallel_calculation(ground_truth[mk],M,alpha,beta)
    a['structure'] = ground_truth[mk]
    a['result'] = a_s
    file_path = './result_set/nodes_3000_3/%s.pkl' % mk
    
    with open(file_path, 'wb') as f:
        pickle.dump(a, f)
    del a, a_s
    gc.collect()

# repeating 100 times to get the ratio of accurate inference

In [ ]:
file_path = './result_set/ground_truth_3000_8.pkl'
with open(file_path, 'rb') as f:
    ground_truth = pickle.load(f)

In [ ]:
M=10
alpha=0.5
beta=0.3
# result1 = dict()
for mk in [list(ground_truth.keys())[0],list(ground_truth.keys())[5]]:
    a = dict()
    a_s = parallel_calculation_100(ground_truth[mk],M,alpha,beta)
    a['structure'] = ground_truth[mk]
    a['result'] = a_s
    file_path = './result_set/nodes_3000_8_repeat_100/%s.pkl' % mk
    
    with open(file_path, 'wb') as f:
        pickle.dump(a, f)
    del a, a_s
    gc.collect()

# empirical network

In [7]:
def SearchFiles(directory, fileType):      
    fileList=[]    
    for root, subDirs, files in os.walk(directory):
        for fileName in files:
            if fileName.endswith(fileType):
                fileList.append(os.path.join(root,fileName))
    return fileList

In [8]:
real_data_set = dict()
directory = './result_set/empirical_network/data_set/'
fileType= 'edges.csv'
fileList = SearchFiles(directory, fileType)
print(len(fileList))

k = 0
for i in fileList:
    df = pd.read_csv(i)
    G = nx.from_pandas_edgelist(df, df.columns[0], df.columns[1])
    isolated_nodes = list(nx.isolates(G))
    G.remove_nodes_from(isolated_nodes)
    real_data_set['%s' % k] = G
    print(G.is_directed())
    k = k+1

fileType= '.txt'
fileList1 = SearchFiles(directory, fileType)
print(fileList1)

for i in fileList1:
    G = nx.read_edgelist(i, nodetype=int, create_using=nx.Graph())
    isolated_nodes = list(nx.isolates(G))
    G.remove_nodes_from(isolated_nodes)
    real_data_set['%s' % k] = G
    print(G.is_directed())
    k = k+1

37
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
['./result_set/empirical_network/data_set/friendships-hamster/friendships-hamster.txt', './result_set/empirical_network/data_set/maayan-Stelzl/maayan-Stelzl.txt', './result_set/empirical_network/data_set/mouse-kasthuri/bn-mouse-kasthuri_graph_v4.txt']
False
False
False


In [10]:
fL = fileList + fileList1

In [18]:
import re

def extract_between_slashes(s):
    # 使用正则表达式匹配第二个斜杠到第三个斜杠之间的字符
    match = re.search(r'\/[^\/]+\/[^\/]+\/[^\/]+\/([^\/]+)\/', s)
    if match:
        # 获取匹配到的部分，并去掉.csv
        result = match.group(1).replace('.csv', '')
        return result
    return None

In [19]:
real_data_information = pd.DataFrame(columns=['Name', 'Network size', 'Mean degree', 'Degree heterogeneity'])
degree_hete = degree_heterogeneity(real_data_set)
mean_de = read_mean_degree(real_data_set)
for i in range(len(fL)):
    real_data_information.loc[i] = [extract_between_slashes(fL[i]), real_data_set['%s' % i].number_of_nodes(), mean_de[i], degree_hete[i]]

In [20]:
real_data_information

,Name,Network size,Mean degree,Degree heterogeneity
0,19971108,3015,3.674295,83.163943
1,394,1096,8.874088,8.750218
2,481743,1343,15.946389,33.368609
3,4950,1246,7.054575,8.534731
4,aae,926,5.064795,26.484459
5,afu,909,4.528053,24.763006
6,ana,1362,5.287812,43.939069
7,ape,805,4.698137,21.555168
8,atu,1593,5.490270,55.083478
9,bible_nouns,1773,10.300056,31.018419


In [21]:
ground_truth = dict()
ground_truth['real_1008'] = real_data_set
len(real_data_set)

40

In [ ]:
M=10
alpha=0.5
beta=0.3
# result1 = dict()
# ground_truth = true_hete_network
for mk in list(ground_truth.keys()):
    a = dict()
    a_s = parallel_calculation(ground_truth[mk],M,alpha,beta)
    a['structure'] = ground_truth[mk]
    a['result'] = a_s
    file_path = './result_set/empirical_network/%s.pkl' % mk
    
    with open(file_path, 'wb') as f:
        pickle.dump(a, f)
    del a, a_s
    gc.collect()